In [1]:
#download repo
!git clone https://github.com/victorchall/EveryDream.git
# Set working directory
%cd EveryDream

c:\Users\texas\Desktop\School\SceneFusion\AnimateDiff-SceneFusion\data\EveryDream


fatal: destination path 'EveryDream' already exists and is not an empty directory.


In [2]:

# install requirements
# !pip install torch=='1.12.1+cu113' 'torchvision==0.13.1+cu113' --extra-index-url https://download.pytorch.org/whl/cu113
!pip install pandas>='1.3.5'
!git clone https://github.com/salesforce/BLIP scripts/BLIP
!pip install timm
!pip install fairscale
# !pip install transformers=='4.19.2'
!pip install timm
!pip install aiofiles
!pip install aiohttp
!pip install opencv-python

fatal: destination path 'scripts/BLIP' already exists and is not an empty directory.


In [3]:
# !pip uninstall pillow
# !pip install pillow

In [4]:
%cd ..

c:\Users\texas\Desktop\School\SceneFusion\AnimateDiff-SceneFusion\data


In [5]:
%ls

 Volume in drive C is OS
 Volume Serial Number is 9C50-ADC8

 Directory of c:\Users\texas\Desktop\School\SceneFusion\AnimateDiff-SceneFusion\data

11/16/2023  02:50 PM    <DIR>          .
11/16/2023  02:30 PM    <DIR>          ..
11/15/2023  12:32 PM    <DIR>          .cache
11/16/2023  02:52 PM            31,383 data.ipynb
11/15/2023  12:27 PM    <DIR>          EveryDream
11/15/2023  01:23 PM    <DIR>          output
11/15/2023  01:57 PM               262 output.csv
11/15/2023  11:56 AM    <DIR>          raw
               2 File(s)         31,645 bytes
               6 Dir(s)  170,972,418,048 bytes free


In [1]:
# !python EveryDream/scripts/auto_caption.py --img_dir raw/1 --out_dir output 

In [2]:
from torchvision import transforms
SIZE = 256

c:\Users\texas\anaconda3\envs\animatediff\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_image(raw_image, device):
    transform = transforms.Compose([
        #transforms.CenterCrop(SIZE),
        transforms.Resize((SIZE, SIZE), interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    image = transform(raw_image).unsqueeze(0).to(device)
    return image

In [4]:
import os
import sys
import torch
import aiofiles
import io

if not os.path.exists("EveryDream/scripts/BLIP"):
    print("BLIP not found, cloning BLIP repo")
    subprocess.run(["git", "clone", "https://github.com/salesforce/BLIP", "EveryDream/scripts/BLIP"])
blip_path = "EveryDream/scripts/BLIP/"
sys.path.append(blip_path)

import models.blip

config_path = "EveryDream/scripts/BLIP/configs/med_config.json"

cache_folder = ".cache"
model_cache_path = ".cache/model_base_caption_capfilt_large.pth"

blip_decoder = models.blip.blip_decoder(pretrained=model_cache_path, image_size=SIZE, vit='base', med_config=config_path)
blip_decoder.eval()

print(f"loading model to cuda")

blip_decoder = blip_decoder.to(torch.device("cuda"))
    

reshape position embedding from 576 to 256
load checkpoint from .cache/model_base_caption_capfilt_large.pth
loading model to cuda


In [10]:
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
import cv2

testing = True

count = os.listdir(f'raw')
print(count)
minFileCount = 1000
folder = 1

# create csv file
import pandas as pd
df = pd.DataFrame(columns=['videoid', 'name', 'page_dir'])

!rm -rf output

for i,dirname in enumerate(count):
    # get list of images from raw/{i} folder
    images = os.listdir(f'raw/{dirname}')
    
    if len(images) < minFileCount:
        minFileCount = len(images)
        folder = dirname
    
    # rename images to 1.jpg, 2.jpg, 3.jpg, etc
    for j in range(len(images)):
        os.rename(f'raw/{dirname}/{images[j]}', f'raw/{dirname}/{j}.png')
        images[j] = f'raw/{dirname}/{j}.png'
    
    img_file_name = images[0]
    
    async with aiofiles.open(img_file_name, "rb") as input_file:
        print("working image: ", img_file_name)

        image_bin = await input_file.read()
        image = Image.open(io.BytesIO(image_bin))

        if not image.mode == "RGB":
            image = image.convert("RGB")

        image = load_image(image, device=torch.device("cuda"))
        # caption the image
        captions = blip_decoder.generate(image, sample=False, num_beams=16, min_length=22, max_length=48, repetition_penalty=1.0)
        caption = captions[0]
        
        # save caption as name to csv file
        df.loc[len(df)] = [i, caption, f'raw/{dirname}']
        
        # print(caption)
        
        # turn images into video
        # Get the images
        images = [cv2.imread(image_path) for image_path in images]
        # crop image to center square shape using the shortest side
        images = [image[:, int((image.shape[1] - image.shape[0]) / 2):int((image.shape[1] + image.shape[0]) / 2)] for image in images]
        # resize images to 256x256
        images = [cv2.resize(image, (256, 256)) for image in images]
        
        !rm -rf output\tmp
        !mkdir output\tmp
        
        # save frames to output/tmp folder for debugging
        for j,image in enumerate(images):
            cv2.imwrite(f'output/tmp/{j}.png', image)
        
        
        
        !ffmpeg -r 1 -i output/tmp/%d.png -c:v libx264 -vf fps=1 output/{i}.mp4
        

        # # Create a video writer
        # video_writer = cv2.VideoWriter(f'output/{i}.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 1, (256, 256))
        
        # # fix moov atom not found error
        # # https://stackoverflow.com/questions/30140432/how-to-fix-moov-atom-not-found

        # # Write the images to the video writer
        # for image in images:
        #     video_writer.write(image)
            
            
        # # Close the writer
        # video_writer.release()
        
            
        if testing and i > 11:
            break
            
print("minFileCount: ", minFileCount)
            
# save csv file
df.to_csv('output.csv', index=False)

['3dDS1', '3dDS10', '3dDS11', '3dDS12', '3dDS13', '3dDS14', '3dDS15', '3dDS16', '3dDS17', '3dDS18', '3dDS19', '3dDS2', '3dDS20', '3dDS21', '3dDS22', '3dDS23', '3dDS24', '3dDS25', '3dDS26', '3dDS27', '3dDS28', '3dDS29', '3dDS3', '3dDS30', '3dDS31', '3dDS32', '3dDS33', '3dDS34', '3dDS35', '3dDS36', '3dDS4', '3dDS5', '3dDS6', '3dDS7', '3dDS8', '3dDS9']
working image:  raw/3dDS1/0.png


ffmpeg version N-110126-g4d216654ca-20230330 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (crosstool-NG 1.25.0.152_89671bf)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libass --enable-libbluray --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librist --enable-libssh --enable-

working image:  raw/3dDS10/0.png


ffmpeg version N-110126-g4d216654ca-20230330 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (crosstool-NG 1.25.0.152_89671bf)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libass --enable-libbluray --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librist --enable-libssh --enable-

working image:  raw/3dDS11/0.png


...

ffmpeg version N-110126-g4d216654ca-20230330 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (crosstool-NG 1.25.0.152_89671bf)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libass --enable-libbluray --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librist --enable-libssh --enable-

working image:  raw/3dDS2/0.png


ffmpeg version N-110126-g4d216654ca-20230330 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (crosstool-NG 1.25.0.152_89671bf)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libass --enable-libbluray --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librist --enable-libssh --enable-

working image:  raw/3dDS20/0.png
minFileCount:  5


ffmpeg version N-110126-g4d216654ca-20230330 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (crosstool-NG 1.25.0.152_89671bf)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libass --enable-libbluray --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librist --enable-libssh --enable-

In [6]:
!ffmpeg -i output/0.mp4

ffmpeg version N-110126-g4d216654ca-20230330 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (crosstool-NG 1.25.0.152_89671bf)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libass --enable-libbluray --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librist --enable-libssh --enable-